In [1]:
import pandas as pd

# Define the column names
columns = [
    "UnderwrittenCoverID", "PolicyID", "TransactionMonth", "IsVATRegistered", "Citizenship", "LegalType", 
    "Title", "Language", "Bank", "AccountType", "MaritalStatus", "Gender", "Country", "Province", "PostalCode", 
    "MainCrestaZone", "SubCrestaZone", "ItemType", "mmcode", "VehicleType", "RegistrationYear", "make", "Model", 
    "Cylinders", "cubiccapacity", "kilowatts", "bodytype", "NumberOfDoors", "VehicleIntroDate", 
    "CustomValueEstimate", "AlarmImmobiliser", "TrackingDevice", "CapitalOutstanding", "NewVehicle", "WrittenOff", 
    "Rebuilt", "Converted", "CrossBorder", "NumberOfVehiclesInFleet", "SumInsured", "TermFrequency", 
    "CalculatedPremiumPerTerm", "ExcessSelected", "CoverCategory", "CoverType", "CoverGroup", "Section", 
    "Product", "StatutoryClass", "StatutoryRiskType", "TotalPremium", "TotalClaims"
]

# Read the file directly (adjust the path as necessary)
df = pd.read_csv(r'..\MachineLearningRating.txt', sep='|', header=None, names=columns, low_memory=False)

# Print the first few rows of the DataFrame
print(df.head(1))


   UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0               145249     12827  2015-03-01 00:00:00             True   

  Citizenship          LegalType Title Language                 Bank  \
0              Close Corporation    Mr  English  First National Bank   

       AccountType  ...         ExcessSelected CoverCategory   CoverType  \
0  Current account  ...  Mobility - Windscreen    Windscreen  Windscreen   

             CoverGroup              Section                          Product  \
0  Comprehensive - Taxi  Motor Comprehensive  Mobility Metered Taxis: Monthly   

  StatutoryClass StatutoryRiskType  TotalPremium TotalClaims  
0     Commercial     IFRS Constant     21.929825         0.0  

[1 rows x 52 columns]


In [5]:
import pandas as pd

# Assuming 'data' is your DataFrame
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])


Bank                        145961
AccountType                  40232
MaritalStatus                 8259
Gender                        9536
mmcode                         552
VehicleType                    552
make                           552
Model                          552
Cylinders                      552
cubiccapacity                  552
kilowatts                      552
bodytype                       552
NumberOfDoors                  552
VehicleIntroDate               552
CustomValueEstimate         779642
CapitalOutstanding               2
NewVehicle                  153295
WrittenOff                  641901
Rebuilt                     641901
Converted                   641901
CrossBorder                 999400
NumberOfVehiclesInFleet    1000098
dtype: int64


In [6]:
# Impute numerical data with median
numerical_columns = df.select_dtypes(include='number')
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].median())

# Impute categorical data with mode
categorical_columns = df.select_dtypes(include='object')
for category_column in categorical_columns:
    df[category_column] = df[category_column].fillna(df[category_column].mode()[0])


C:\Users\CraftSoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [9]:
from datetime import datetime

current_year = datetime.now().year
df['VehicleAge'] = current_year - df['RegistrationYear']
df['VehicleAgeGroup'] = pd.cut(df['VehicleAge'], bins=[0, 1, 5, 10, 100], labels=['New', '1-5 years', '6-10 years', 'Over 10 years'])
df['PremiumToClaimsRatio'] = df['TotalPremium'] / df['TotalClaims']


In [10]:
df_encoded = pd.get_dummies(df, columns=['Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'make', 'Model', 'bodytype', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType'])


In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['IsVATRegistered'] = le.fit_transform(df['IsVATRegistered'])


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# Load your DataFrame (replace this with your actual data loading method)
# df = pd.read_csv('your_file.csv')  # Example of loading data

# Check the columns in your DataFrame
print("Original DataFrame Columns:", df.columns)

# Data Preparation
# Fill missing values
# Impute numerical data with median
numerical_columns = df.select_dtypes(include='number')
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].median())

# Impute categorical data with mode
categorical_columns = df.select_dtypes(include='object')
for category_column in categorical_columns:
    df[category_column] = df[category_column].fillna(df[category_column].mode()[0])

# if 'RegistrationYear' in df.columns:
#     df['RegistrationYear'] = df['RegistrationYear'].fillna(df['RegistrationYear'].median())
# else:
#     print("Column 'RegistrationYear' not found in DataFrame")

# if 'Bank' in df.columns:
#     df['Bank'] = df['Bank'].fillna(df['Bank'].mode()[0])
# else:
#     print("Column 'Bank' not found in DataFrame")

# Feature Engineering and Encoding
categorical_columns = [
    'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
    'MaritalStatus', 'Gender', 'Country', 'Province', 'MainCrestaZone',
    'SubCrestaZone', 'ItemType', 'make', 'Model', 'bodytype', 'CoverCategory',
    'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType'
]

# Check which categorical columns are present in the DataFrame
existing_categorical_columns = [col for col in categorical_columns if col in df.columns]
print("Categorical Columns Present:", existing_categorical_columns)

# Create a list of columns to exclude from encoding (including the target variable)
exclude_columns = ['TotalPremium', 'TotalClaims']

# Ensure target variable is not encoded
columns_to_encode = [col for col in existing_categorical_columns if col not in exclude_columns]

print("Columns to encode:", columns_to_encode)

if columns_to_encode:
    # Encode categorical columns if they exist
    df_encoded = pd.get_dummies(df, columns=columns_to_encode, drop_first=False)
else:
    print("No categorical columns found to encode.")
    df_encoded = df.copy()  # If no categorical columns, use the original DataFrame

# Check the columns after encoding
print("Columns after encoding:", df_encoded.columns)

# Ensure the target variable 'TotalPremium' is still present
if 'TotalPremium' in df_encoded.columns:
    X = df_encoded.drop(['TotalPremium', 'TotalClaims'], axis=1, errors='ignore')
    y = df_encoded['TotalPremium']
else:
    raise ValueError("Target variable 'TotalPremium' not found in DataFrame")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle NaN values in training data
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())  # Ensure the test set has the same NaN handling as the training set
y_train = y_train.fillna(y_train.mean())
y_test = y_test.fillna(y_train.mean())  # Ensure the test set has the same NaN handling as the training set

# Modeling Techniques

# Function to evaluate models
def evaluate_model(model_name, y_test, predictions):
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    print(f"\n{model_name} Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R² Score: {r2}")

# 1. Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
evaluate_model("Linear Regression", y_test, lr_predictions)

# 2. Decision Tree Regressor
dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)
evaluate_model("Decision Tree Regressor", y_test, dt_predictions)

# 3. Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
evaluate_model("Random Forest Regressor", y_test, rf_predictions)

# 4. XGBoost Regressor
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
evaluate_model("XGBoost Regressor", y_test, xgb_predictions)

# Feature Importance Analysis (for Random Forest and XGBoost)
if hasattr(rf_model, 'feature_importances_'):
    importances_rf = rf_model.feature_importances_
    print("\nFeature Importances (Random Forest):")
    for feature, importance in zip(X.columns, importances_rf):
        print(f"{feature}: {importance}")

if hasattr(xgb_model, 'feature_importances_'):
    importances_xgb = xgb_model.feature_importances_
    print("\nFeature Importances (XGBoost):")
    for feature, importance in zip(X.columns, importances_xgb):
        print(f"{feature}: {importance}")


Original DataFrame Columns: Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims'],
      dtype='object')


C:\Users\CraftSoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Categorical Columns Present: ['Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'make', 'Model', 'bodytype', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType']
Columns to encode: ['Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'make', 'Model', 'bodytype', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType']
Columns after encoding: Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'PostalCode', 'mmcode', 'VehicleType',
       'RegistrationYear', 'Cylinders', 'cubiccapacity',
       ...
       'Section_Motor Comprehensive', 'Section_Optional Extended Covers',
       'Section_Standalone passenger liability',
  

MemoryError: Unable to allocate 13.7 MiB for an array with shape (9, 200020) and data type float64

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import numpy as np

# Check the columns in your DataFrame
print("Original DataFrame Columns:", df.columns)

# Data Preparation
# Fill missing values
# Impute numerical data with median
numerical_columns = df.select_dtypes(include='number')
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].median())

# Impute categorical data with mode
categorical_columns = df.select_dtypes(include='object')
for category_column in categorical_columns:
    df[category_column] = df[category_column].fillna(df[category_column].mode()[0])
# if 'RegistrationYear' in df.columns:
#     df['RegistrationYear'] = df['RegistrationYear'].fillna(df['RegistrationYear'].median())
# else:
#     print("Column 'RegistrationYear' not found in DataFrame")

# if 'Bank' in df.columns:
#     df['Bank'] = df['Bank'].fillna(df['Bank'].mode()[0])
# else:
#     print("Column 'Bank' not found in DataFrame")

# Define categorical columns
categorical_columns = [
    'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType',
    'MaritalStatus', 'Gender', 'Country', 'Province', 'MainCrestaZone',
    'SubCrestaZone', 'ItemType', 'make', 'Model', 'bodytype', 'CoverCategory',
    'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType'
]

# Check which categorical columns are present in the DataFrame
existing_categorical_columns = [col for col in categorical_columns if col in df.columns]
print("Categorical Columns Present:", existing_categorical_columns)

# Use label encoding for columns with high cardinality and one-hot encoding for others
high_cardinality_columns = [col for col in existing_categorical_columns if df[col].nunique() > 10]
low_cardinality_columns = [col for col in existing_categorical_columns if col not in high_cardinality_columns]

# Label Encoding for high cardinality columns
label_encoder = LabelEncoder()
for col in high_cardinality_columns:
    df[col] = label_encoder.fit_transform(df[col].astype(str))

# One-Hot Encoding for low cardinality columns
if low_cardinality_columns:
    df = pd.get_dummies(df, columns=low_cardinality_columns, drop_first=True)

# Ensure the target variable 'TotalPremium' is still present
if 'TotalPremium' in df.columns:
    X = df.drop(['TotalPremium', 'TotalClaims'], axis=1, errors='ignore')
    y = df['TotalPremium']
else:
    raise ValueError("Target variable 'TotalPremium' not found in DataFrame")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle NaN values in training data
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean())  # Ensure the test set has the same NaN handling as the training set
y_train = y_train.fillna(y_train.mean())
y_test = y_test.fillna(y_train.mean())  # Ensure the test set has the same NaN handling as the training set

# Modeling Techniques
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost Regressor": XGBRegressor()
}

# Function to evaluate models
def evaluate_model(model_name, y_test, predictions):
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    print(f"\n{model_name} Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R² Score: {r2}")

# Training and evaluating each model
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    evaluate_model(model_name, y_test, predictions)

# Feature Importance Analysis (for Random Forest and XGBoost)
for model_name, model in models.items():
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        print(f"\nFeature Importances ({model_name}):")
        for feature, importance in zip(X.columns, importances):
            print(f"{feature}: {importance}")
